# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

W0815 19:37:28.454000 3735441 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0815 19:37:28.454000 3735441 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-15 19:37:29] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=38346, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False,

[2025-08-15 19:37:30] Using default HuggingFace chat template with detected content format: string


W0815 19:37:36.949000 3736632 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0815 19:37:36.949000 3736632 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0815 19:37:36.951000 3736631 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0815 19:37:36.951000 3736631 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-15 19:37:38] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-15 19:37:38] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=19.27 GB): 100%|██████████| 3/3 [00:00<00:00,  6.85it/s]
[2025-08-15 19:37:45] Capture cuda graph end. Time elapsed: 1.77 s. mem usage=0.38 GB. avail mem=19.26 GB.


[2025-08-15 19:37:45] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=18.76 GB


[2025-08-15 19:37:46] INFO:     Started server process [3735441]
[2025-08-15 19:37:46] INFO:     Waiting for application startup.
[2025-08-15 19:37:46] INFO:     Application startup complete.
[2025-08-15 19:37:46] INFO:     Uvicorn running on http://0.0.0.0:38346 (Press CTRL+C to quit)
[2025-08-15 19:37:46] INFO:     127.0.0.1:41974 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-15 19:37:47] INFO:     127.0.0.1:41980 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-15 19:37:47] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:37:47] INFO:     127.0.0.1:41988 - "POST /generate HTTP/1.1" 200 OK
[2025-08-15 19:37:47] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-15 19:37:51] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:37:53] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.44, #queue-req: 0, 


[2025-08-15 19:37:53] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, cuda graph: True, gen throughput (token/s): 168.38, #queue-req: 0, 


[2025-08-15 19:37:53] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.85, #queue-req: 0, 


[2025-08-15 19:37:53] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, cuda graph: True, gen throughput (token/s): 103.99, #queue-req: 0, 


...

[2025-08-15 19:37:54] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.15, #queue-req: 0, 


[2025-08-15 19:37:54] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, cuda graph: True, gen throughput (token/s): 156.46, #queue-req: 0, 


[2025-08-15 19:37:55] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.17, #queue-req: 0, 


[2025-08-15 19:37:55] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, cuda graph: True, gen throughput (token/s): 158.36, #queue-req: 0, 


[2025-08-15 19:37:55] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: True, gen throughput (token/s): 161.95, #queue-req: 0, 
[2025-08-15 19:37:55] INFO:     127.0.0.1:41994 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-15 19:37:55] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:37:56] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, cuda graph: True, gen throughput (token/s): 135.61, #queue-req: 0, 


[2025-08-15 19:37:56] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, cuda graph: True, gen throughput (token/s): 168.33, #queue-req: 0, 


[2025-08-15 19:37:56] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, cuda graph: True, gen throughput (token/s): 151.00, #queue-req: 0, 


[2025-08-15 19:37:56] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, cuda graph: True, gen throughput (token/s): 151.80, #queue-req: 0, 


[2025-08-15 19:37:57] Decode batch. #running-req: 1, #token: 216, token usage: 0.01, cuda graph: True, gen throughput (token/s): 154.87, #queue-req: 0, 


[2025-08-15 19:37:57] Decode batch. #running-req: 1, #token: 256, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.55, #queue-req: 0, 


[2025-08-15 19:37:57] Decode batch. #running-req: 1, #token: 296, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.19, #queue-req: 0, 


[2025-08-15 19:37:57] Decode batch. #running-req: 1, #token: 336, token usage: 0.02, cuda graph: True, gen throughput (token/s): 124.86, #queue-req: 0, 


[2025-08-15 19:37:58] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, cuda graph: True, gen throughput (token/s): 100.49, #queue-req: 0, 
[2025-08-15 19:37:58] INFO:     127.0.0.1:41994 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-15 19:37:58] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:37:58] Decode batch. #running-req: 1, #token: 61, token usage: 0.00, cuda graph: True, gen throughput (token/s): 147.41, #queue-req: 0, 


[2025-08-15 19:37:58] Decode batch. #running-req: 1, #token: 101, token usage: 0.00, cuda graph: True, gen throughput (token/s): 170.26, #queue-req: 0, 


[2025-08-15 19:37:58] Decode batch. #running-req: 1, #token: 141, token usage: 0.01, cuda graph: True, gen throughput (token/s): 169.87, #queue-req: 0, 


[2025-08-15 19:37:59] Decode batch. #running-req: 1, #token: 181, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.71, #queue-req: 0, 


...

[2025-08-15 19:38:02] Decode batch. #running-req: 1, #token: 581, token usage: 0.03, cuda graph: True, gen throughput (token/s): 95.19, #queue-req: 0, 


[2025-08-15 19:38:02] Decode batch. #running-req: 1, #token: 621, token usage: 0.03, cuda graph: True, gen throughput (token/s): 91.53, #queue-req: 0, 


[2025-08-15 19:38:03] Decode batch. #running-req: 1, #token: 661, token usage: 0.03, cuda graph: True, gen throughput (token/s): 90.87, #queue-req: 0, 


[2025-08-15 19:38:03] Decode batch. #running-req: 1, #token: 701, token usage: 0.03, cuda graph: True, gen throughput (token/s): 167.11, #queue-req: 0, 


[2025-08-15 19:38:03] Decode batch. #running-req: 1, #token: 741, token usage: 0.04, cuda graph: True, gen throughput (token/s): 154.64, #queue-req: 0, 
[2025-08-15 19:38:03] INFO:     127.0.0.1:41994 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-15 19:38:03] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:38:03] Decode batch. #running-req: 1, #token: 31, token usage: 0.00, cuda graph: True, gen throughput (token/s): 137.06, #queue-req: 0, 


[2025-08-15 19:38:04] Decode batch. #running-req: 1, #token: 71, token usage: 0.00, cuda graph: True, gen throughput (token/s): 149.49, #queue-req: 0, 


[2025-08-15 19:38:04] Decode batch. #running-req: 1, #token: 111, token usage: 0.01, cuda graph: True, gen throughput (token/s): 147.90, #queue-req: 0, 
[2025-08-15 19:38:04] INFO:     127.0.0.1:41994 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-15 19:38:05] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-15 19:38:05] Decode batch. #running-req: 1, #token: 491, token usage: 0.02, cuda graph: True, gen throughput (token/s): 42.20, #queue-req: 0, 


[2025-08-15 19:38:05] Decode batch. #running-req: 1, #token: 531, token usage: 0.03, cuda graph: True, gen throughput (token/s): 160.50, #queue-req: 0, 


[2025-08-15 19:38:05] Decode batch. #running-req: 1, #token: 571, token usage: 0.03, cuda graph: True, gen throughput (token/s): 161.18, #queue-req: 0, 


[2025-08-15 19:38:06] Decode batch. #running-req: 1, #token: 611, token usage: 0.03, cuda graph: True, gen throughput (token/s): 161.06, #queue-req: 0, 


[2025-08-15 19:38:06] Decode batch. #running-req: 1, #token: 651, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.16, #queue-req: 0, 


[2025-08-15 19:38:06] Decode batch. #running-req: 1, #token: 691, token usage: 0.03, cuda graph: True, gen throughput (token/s): 158.53, #queue-req: 0, 


[2025-08-15 19:38:06] Decode batch. #running-req: 1, #token: 731, token usage: 0.04, cuda graph: True, gen throughput (token/s): 162.22, #queue-req: 0, 
[2025-08-15 19:38:06] INFO:     127.0.0.1:41994 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-08-15 19:38:07] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:38:07] Decode batch. #running-req: 1, #token: 62, token usage: 0.00, cuda graph: True, gen throughput (token/s): 52.15, #queue-req: 0, 


[2025-08-15 19:38:07] Decode batch. #running-req: 1, #token: 102, token usage: 0.00, cuda graph: True, gen throughput (token/s): 170.46, #queue-req: 0, 


[2025-08-15 19:38:08] Decode batch. #running-req: 1, #token: 142, token usage: 0.01, cuda graph: True, gen throughput (token/s): 169.89, #queue-req: 0, 


[2025-08-15 19:38:08] Decode batch. #running-req: 1, #token: 182, token usage: 0.01, cuda graph: True, gen throughput (token/s): 124.58, #queue-req: 0, 


...

[2025-08-15 19:38:09] Decode batch. #running-req: 1, #token: 262, token usage: 0.01, cuda graph: True, gen throughput (token/s): 91.13, #queue-req: 0, 


[2025-08-15 19:38:09] Decode batch. #running-req: 1, #token: 302, token usage: 0.01, cuda graph: True, gen throughput (token/s): 160.19, #queue-req: 0, 


[2025-08-15 19:38:09] Decode batch. #running-req: 1, #token: 342, token usage: 0.02, cuda graph: True, gen throughput (token/s): 152.44, #queue-req: 0, 


[2025-08-15 19:38:09] Decode batch. #running-req: 1, #token: 382, token usage: 0.02, cuda graph: True, gen throughput (token/s): 152.38, #queue-req: 0, 


[2025-08-15 19:38:10] Decode batch. #running-req: 1, #token: 422, token usage: 0.02, cuda graph: True, gen throughput (token/s): 155.84, #queue-req: 0, 
[2025-08-15 19:38:10] INFO:     127.0.0.1:57284 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down. First, I need to identify what the capital of France is. I know that Paris is the capital, so that\'s straightforward. \n\nNext, I need to find the population. I remember that Paris is a major city, so its population is quite large. I think it\'s over 3 million, but I\'m not exactly sure of the exact number. Maybe I should double-check that. \n\nWait, I recall that the population figure can vary depending on the source and the year. The user didn\'t specify a particular year, so I should probably go with the most recent estimate. I believe the population is around 3,500,000 as of 2023. \n\nNow, I need to structure this

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-08-15 19:38:10] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:38:10] Decode batch. #running-req: 1, #token: 57, token usage: 0.00, cuda graph: True, gen throughput (token/s): 156.01, #queue-req: 0, 


[2025-08-15 19:38:10] Decode batch. #running-req: 1, #token: 97, token usage: 0.00, cuda graph: True, gen throughput (token/s): 169.71, #queue-req: 0, 


[2025-08-15 19:38:10] Decode batch. #running-req: 1, #token: 137, token usage: 0.01, cuda graph: True, gen throughput (token/s): 157.89, #queue-req: 0, 


[2025-08-15 19:38:11] Decode batch. #running-req: 1, #token: 177, token usage: 0.01, cuda graph: True, gen throughput (token/s): 154.81, #queue-req: 0, 


[2025-08-15 19:38:11] Decode batch. #running-req: 1, #token: 217, token usage: 0.01, cuda graph: True, gen throughput (token/s): 142.86, #queue-req: 0, 


[2025-08-15 19:38:11] Decode batch. #running-req: 1, #token: 257, token usage: 0.01, cuda graph: True, gen throughput (token/s): 143.29, #queue-req: 0, 


[2025-08-15 19:38:12] Decode batch. #running-req: 1, #token: 297, token usage: 0.01, cuda graph: True, gen throughput (token/s): 153.32, #queue-req: 0, 


[2025-08-15 19:38:12] Decode batch. #running-req: 1, #token: 337, token usage: 0.02, cuda graph: True, gen throughput (token/s): 166.38, #queue-req: 0, 


[2025-08-15 19:38:12] Decode batch. #running-req: 1, #token: 377, token usage: 0.02, cuda graph: True, gen throughput (token/s): 168.09, #queue-req: 0, 
[2025-08-15 19:38:12] INFO:     127.0.0.1:57296 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-08-15 19:38:12] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-15 19:38:12] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 20, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-15 19:38:12] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 2, #queue-req: 0, 
[2025-08-15 19:38:12] Decode batch. #running-req: 3, #token: 35, token usage: 0.00, cuda graph: True, gen throughput (token/s): 152.32, #queue-req: 0, 


[2025-08-15 19:38:13] Decode batch. #running-req: 3, #token: 155, token usage: 0.01, cuda graph: True, gen throughput (token/s): 495.41, #queue-req: 0, 


[2025-08-15 19:38:13] Decode batch. #running-req: 3, #token: 275, token usage: 0.01, cuda graph: True, gen throughput (token/s): 450.50, #queue-req: 0, 


[2025-08-15 19:38:13] Decode batch. #running-req: 3, #token: 395, token usage: 0.02, cuda graph: True, gen throughput (token/s): 473.38, #queue-req: 0, 


[2025-08-15 19:38:14] Decode batch. #running-req: 3, #token: 515, token usage: 0.03, cuda graph: True, gen throughput (token/s): 321.24, #queue-req: 0, 


[2025-08-15 19:38:14] Decode batch. #running-req: 1, #token: 219, token usage: 0.01, cuda graph: True, gen throughput (token/s): 177.07, #queue-req: 0, 


[2025-08-15 19:38:14] Decode batch. #running-req: 1, #token: 259, token usage: 0.01, cuda graph: True, gen throughput (token/s): 141.10, #queue-req: 0, 
[2025-08-15 19:38:14] INFO:     127.0.0.1:40056 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThe user has provided the correct information that the capital of France is Paris. However, the user's request was to provide the information of the capital of France, which is a bit ambiguous. It could be interpreted as wanting more detailed information about Paris, such as its history, culture, landmarks, or government. Alternatively, it might mean they want to confirm that Paris is indeed the capital, which they already know. \n\nTo better assist the user, it would be helpful to ask for clarification. If they want more details about Paris, I can provide a comprehensive overview. If they just need confirmation, then the current response suffices. \n\nAdditionally, I can offer to provide more information o

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-08-15 19:38:14] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:38:15] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: True, gen throughput (token/s): 155.26, #queue-req: 0, 


[2025-08-15 19:38:15] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: True, gen throughput (token/s): 122.81, #queue-req: 0, 


[2025-08-15 19:38:15] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: True, gen throughput (token/s): 160.75, #queue-req: 0, 


[2025-08-15 19:38:15] Decode batch. #running-req: 1, #token: 160, token usage: 0.01, cuda graph: True, gen throughput (token/s): 141.98, #queue-req: 0, 


...

[2025-08-15 19:38:26] Decode batch. #running-req: 1, #token: 1880, token usage: 0.09, cuda graph: True, gen throughput (token/s): 163.68, #queue-req: 0, 


[2025-08-15 19:38:27] Decode batch. #running-req: 1, #token: 1920, token usage: 0.09, cuda graph: True, gen throughput (token/s): 163.70, #queue-req: 0, 


[2025-08-15 19:38:27] Decode batch. #running-req: 1, #token: 1960, token usage: 0.10, cuda graph: True, gen throughput (token/s): 163.35, #queue-req: 0, 


[2025-08-15 19:38:27] Decode batch. #running-req: 1, #token: 2000, token usage: 0.10, cuda graph: True, gen throughput (token/s): 162.35, #queue-req: 0, 


[2025-08-15 19:38:27] Decode batch. #running-req: 1, #token: 2040, token usage: 0.10, cuda graph: True, gen throughput (token/s): 145.30, #queue-req: 0, 
[2025-08-15 19:38:27] INFO:     127.0.0.1:40058 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-08-15 19:38:27] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 19:38:28] Decode batch. #running-req: 1, #token: 49, token usage: 0.00, cuda graph: True, gen throughput (token/s): 131.79, #queue-req: 0, 


[2025-08-15 19:38:28] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, cuda graph: True, gen throughput (token/s): 149.93, #queue-req: 0, 


[2025-08-15 19:38:28] Decode batch. #running-req: 1, #token: 129, token usage: 0.01, cuda graph: True, gen throughput (token/s): 158.70, #queue-req: 0, 


[2025-08-15 19:38:28] Decode batch. #running-req: 1, #token: 169, token usage: 0.01, cuda graph: True, gen throughput (token/s): 148.41, #queue-req: 0, 


...

[2025-08-15 19:38:30] Decode batch. #running-req: 1, #token: 449, token usage: 0.02, cuda graph: True, gen throughput (token/s): 157.34, #queue-req: 0, 


[2025-08-15 19:38:30] Decode batch. #running-req: 1, #token: 489, token usage: 0.02, cuda graph: True, gen throughput (token/s): 148.06, #queue-req: 0, 


[2025-08-15 19:38:31] Decode batch. #running-req: 1, #token: 529, token usage: 0.03, cuda graph: True, gen throughput (token/s): 148.26, #queue-req: 0, 


[2025-08-15 19:38:31] Decode batch. #running-req: 1, #token: 569, token usage: 0.03, cuda graph: True, gen throughput (token/s): 117.08, #queue-req: 0, 


[2025-08-15 19:38:31] Decode batch. #running-req: 1, #token: 609, token usage: 0.03, cuda graph: True, gen throughput (token/s): 145.81, #queue-req: 0, 
[2025-08-15 19:38:31] INFO:     127.0.0.1:47524 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

W0815 19:38:33.531000 3734926 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0815 19:38:33.531000 3734926 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0815 19:38:42.114000 3743317 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0815 19:38:42.114000 3743317 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.40s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.33s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.34s/it]



Capturing batches (bs=4 avail_mem=10.97 GB):   0%|          | 0/3 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=10.85 GB): 100%|██████████| 3/3 [00:00<00:00,  4.88it/s]


### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris. This is a city that's known for its rich history, art, and landmarks. It's situated in the northern part of the country, and is one of the most significant cultural and political centers in Europe. Paris is home to the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and the Tuileries Gardens, among other famous attractions. It has a vibrant and diverse culture, with a mix of French, international, and historical influences. The city is also known for its fashion and cuisine, which are prominent worldwide. Paris is a metropolis, meaning it's a
Prompt: Give me the information of the capital of Germany.
Generated text:  the capital of Germany is Berlin.

Identify the given statement as true or false.

True
False

The given statement is: "The capital of Germany is Berlin." 

I know that the capital of Germany is indeed Berlin. Berlin is the most populous city in Germany and has

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its economic, cultural, and social significance.

800 words, in English, no markdown.

**Summary**

London is a major global city renowned for its economic, cultural, and social significance. It is a global financial center, home to 40% of the world's financial services and a significant cultural and social hub. The city is a multicultural metropolis, home to people from all over the world, and is known for its vibrant nightlife, world-class museums, and landmarks. The city has a rich history and is an important political and military center.

**Question**

Who are some of the prominent people from London?

Please
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, economic significance, cultural significance, and the most beautiful landmarks, including their location, description, and significance.

Please provide a structured answer, with each sec

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Alright, the user is asking for the information and population of France's capital in JSON format. Okay, so first, I need to figure out what exactly they're looking for. The capital of France is definitely Paris, so that's the key point.

They want the information in JSON, which means I need to structure it properly. I should make sure to include key-value pairs. Maybe the main keys are "Country" and "Capital" because those are the two main pieces of information requested.

For the population, I recall that Paris is the most populous city in France but not the country as a whole. I think the population figure is around 2 million. But I should double-check that to be accurate. Maybe it's about 2.2 million as of recent data.

I also need to present this information clearly. So, I'll create a JSON object with 

In [19]:
llm.shutdown()